<a href="https://colab.research.google.com/github/Deolinda1506/MLOPS-1/blob/main/notebook/lung_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>